# Regression Week 5: LASSO (coordinate descent)

In this notebook, you will implement your very own LASSO solver via coordinate descent. You will:
* Write a function to normalize features
* Implement coordinate descent for LASSO
* Explore effects of L1 penalty

# Fire up Turi Create

In [168]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':float, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [169]:
sales = pd.read_csv('kc_house_data.csv', dtype=dtype_dict)

If we want to do any "feature engineering" like creating new features or adjusting existing ones we should do this directly using the SFrames as seen in the first notebook of Week 2. For this notebook, however, we will work with the existing features.

# Import useful functions from previous notebook

As in Week 2, we convert the SFrame into a 2D Numpy array. Copy and paste `get_num_data()` from the second notebook of Week 2.

In [170]:
import numpy as np # note this allows us to refer to numpy as np instead 

In [171]:
def get_numpy_data(data_sale, features, output):
    data_sale['constant'] = 1 # this is how you add a constant column to an SFrame
    # add the column 'constant' to the front of the features list so that we can extract it along with the others:
    features =   ['constant'] + features  # this is how you combine two lists
    #print (features)// print ['sqft_living', 'constant']
    # select the columns of data_SFrame given by the features list into the SFrame features_sframe (now including constant):
    sale_features=data_sale[features]
    #print (sale_features) print all sqft liv +constant
    # the following line will convert the features_SFrame into a numpy matrix:
    feature_matrix = sale_features.to_numpy()
    # assign the column of data_sframe associated with the output to the SArray output_sarray
    out=data_sale[output]
    out_arr=np.array(out)
    # the following will convert the SArray into a numpy array by first converting it to a list
    return(feature_matrix, out_arr)

Also, copy and paste the `predict_output()` function to compute the predictions for an entire matrix of features given the matrix and the weights:

In [172]:
def predict_output(feature_matrix, weights):
    predictions=np.dot(feature_matrix, weights)

    return(predictions)

# Normalize features
In the house dataset, features vary wildly in their relative magnitude: `sqft_living` is very large overall compared to `bedrooms`, for instance. As a result, weight for `sqft_living` would be much smaller than weight for `bedrooms`. This is problematic because "small" weights are dropped first as `l1_penalty` goes up. 

To give equal considerations for all features, we need to **normalize features** as discussed in the lectures: we divide each feature by its 2-norm so that the transformed feature has norm 1.

Let's see how we can do this normalization easily with Numpy: let us first consider a small matrix.

In [173]:
X = np.array([[3.,5.,8.],[4.,12.,15.]])
print (X)
weights = np.array([1., 4.,1.])
pred=predict_output(X,weights)
print(pred)

[[ 3.  5.  8.]
 [ 4. 12. 15.]]
[31. 67.]


Numpy provides a shorthand for computing 2-norms of each column:

In [174]:
norms = np.linalg.norm(X, axis=0) # gives [norm(X[:,0]), norm(X[:,1]), norm(X[:,2])]
print (norms)

[ 5. 13. 17.]


To normalize, apply element-wise division:

In [175]:
print (X / norms) # gives [X[:,0]/norm(X[:,0]), X[:,1]/norm(X[:,1]), X[:,2]/norm(X[:,2])]

[[0.6        0.38461538 0.47058824]
 [0.8        0.92307692 0.88235294]]


Using the shorthand we just covered, write a short function called `normalize_features(feature_matrix)`, which normalizes columns of a given feature matrix. The function should return a pair `(normalized_features, norms)`, where the second item contains the norms of original features. As discussed in the lectures, we will use these norms to normalize the test data in the same way as we normalized the training data. 

In [176]:
def normalize_features(feature_matrix):
    norms = np.linalg.norm(feature_matrix, axis=0)
    normalized_features=(feature_matrix/norms)
    return (normalized_features,norms)

To test the function, run the following:

In [177]:
features, norms = normalize_features(np.array([[3.,6.,9.],[4.,8.,12.]]))
print (features)
# should print
# [[ 0.6  0.6  0.6]
#  [ 0.8  0.8  0.8]]
print (norms)
# should print
# [5.  10.  15.]

[[0.6 0.6 0.6]
 [0.8 0.8 0.8]]
[ 5. 10. 15.]


# Implementing Coordinate Descent with normalized features

We seek to obtain a sparse set of weights by minimizing the LASSO cost function
```
SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|).
```
(By convention, we do not include `w[0]` in the L1 penalty term. We never want to push the intercept to zero.)

The absolute value sign makes the cost function non-differentiable, so simple gradient descent is not viable (you would need to implement a method called subgradient descent). Instead, we will use **coordinate descent**: at each iteration, we will fix all weights but weight `i` and find the value of weight `i` that minimizes the objective. That is, we look for
```
argmin_{w[i]} [ SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|) ]
```
where all weights other than `w[i]` are held to be constant. We will optimize one `w[i]` at a time, circling through the weights multiple times.  
  1. Pick a coordinate `i`
  2. Compute `w[i]` that minimizes the cost function `SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|)`
  3. Repeat Steps 1 and 2 for all coordinates, multiple times

For this notebook, we use **cyclical coordinate descent with normalized features**, where we cycle through coordinates 0 to (d-1) in order, and assume the features were normalized as discussed above. The formula for optimizing each coordinate is as follows:
```
       ┌ (ro[i] + lambda/2)     if ro[i] < -lambda/2
w[i] = ├ 0                      if -lambda/2 <= ro[i] <= lambda/2
       └ (ro[i] - lambda/2)     if ro[i] > lambda/2
```
where
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ].
```

Note that we do not regularize the weight of the constant feature (intercept) `w[0]`, so, for this weight, the update is simply:
```
w[0] = ro[i]
```

## Effect of L1 penalty

Let us consider a simple model with 2 features:

In [178]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)


Don't forget to normalize features:

In [179]:
simple_feature_matrix, norms = normalize_features(simple_feature_matrix)


We assign some random set of initial weights and inspect the values of `ro[i]`:

In [180]:
weights = np.array([1., 4., 1.])

Use `predict_output()` to make predictions on this data.

In [181]:
prediction = predict_output(simple_feature_matrix, weights)


Compute the values of `ro[i]` for each feature in this simple model, using the formula given above, using the formula:
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ]
```

*Hint: You can get a Numpy vector for feature_i using:*
```
simple_feature_matrix[:,i]
```

In [182]:
row_i=[]

for i in range (0,len(weights)):
    r= simple_feature_matrix[:,i]*(output-prediction+weights[i]*simple_feature_matrix[:,i])
   
    row_i.append(np.sum(r))  
print(row_i)  


[79400300.0145229, 87939470.82325175, 80966698.66623947]


***QUIZ QUESTION***

Recall that, whenever `ro[i]` falls between `-l1_penalty/2` and `l1_penalty/2`, the corresponding weight `w[i]` is sent to zero. Now suppose we were to take one step of coordinate descent on either feature 1 or feature 2. What range of values of `l1_penalty` **would not** set `w[1]` zero, but **would** set `w[2]` to zero, if we were to take a step in that coordinate? 

In [183]:
list=[1.4e8,1.64e8,1.73e8,1.9e8,2.3e8]
w_list=[]
w_list_1=[]
for j in range (1,len(list)):
    if(row_i[1]>=-list[j]/2 and row_i[1]<=list[j]/2 ):
        w_list.append(list[j])
    if(row_i[2]>=-list[j]/2 and row_i[2]<=list[j]/2 ):
        w_list_1.append(list[j])
print(w_list,w_list_1)        

[190000000.0, 230000000.0] [164000000.0, 173000000.0, 190000000.0, 230000000.0]


***QUIZ QUESTION***

What range of values of `l1_penalty` would set **both** `w[1]` and `w[2]` to zero, if we were to take a step in that coordinate? 

So we can say that `ro[i]` quantifies the significance of the i-th feature: the larger `ro[i]` is, the more likely it is for the i-th feature to be retained.

## Single Coordinate Descent Step

Using the formula above, implement coordinate descent that minimizes the cost function over a single feature i. Note that the intercept (weight 0) is not regularized. The function should accept feature matrix, output, current weights, l1 penalty, and index of feature to optimize over. The function should return new weight for feature i.

In [184]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    prediction = predict_output(feature_matrix, weights)
    
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    ro_i = (feature_matrix[:,i]*(output-prediction+weights[i]*feature_matrix[:,i])).sum()
    # ro_i=np.sum(ro)
   # print("rss",(output-prediction).sum())
    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i 
        
    elif ro_i < -l1_penalty/2.:
        new_weight_i =(ro_i+(l1_penalty/2.))
       # print("-v",new_weight_i)
    elif ro_i > l1_penalty/2.:
        new_weight_i = (ro_i-(l1_penalty/2.))
       # print(new_weight_i)
    else:
        new_weight_i = 0.
        
    
    return new_weight_i

To test the function, run the following cell:

In [185]:
# should print 0.425558846691
import math
     
print(lasso_coordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],[2./math.sqrt(13),3./math.sqrt(10)]]), 
                                   np.array([1., 1.]), np.array([1., 4.]), 0.1))

0.4255588466910251


## Cyclical coordinate descent 

Now that we have a function that optimizes the cost function over a single coordinate, let us implement cyclical coordinate descent where we optimize coordinates 0, 1, ..., (d-1) in order and repeat.

When do we know to stop? Each time we scan all the coordinates (features) once, we measure the change in weight for each coordinate. If no coordinate changes by more than a specified threshold, we stop.

For each iteration:
1. As you loop over features in order and perform coordinate descent, measure how much each coordinate changes.
2. After the loop, if the maximum change across all coordinates is falls below the tolerance, stop. Otherwise, go back to step 1.

Return weights

**IMPORTANT: when computing a new weight for coordinate i, make sure to incorporate the new weights for coordinates 0, 1, ..., i-1. One good way is to update your weights variable in-place. See following pseudocode for illustration.**
```
for i in range(len(weights)):
    old_weights_i = weights[i] # remember old value of weight[i], as it will be overwritten
    # the following line uses new values for weight[0], weight[1], ..., weight[i-1]
    #     and old values for weight[i], ..., weight[d-1]
    weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
    
    # use old_weights_i to compute change in coordinate
    ...
```

In [186]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    delta=True
    
    while(delta):
        diff=-1000
        for i in range(len(initial_weights)):
            
            old_weights_i = initial_weights[i] # remember old value of weight[i], as it will be overwritten
    # the following line uses new values for weight[0], weight[1], ..., weight[i-1]
    #     and old values for weight[i], ..., weight[d-1]
            initial_weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, initial_weights, l1_penalty)
            temp=(initial_weights[i]-old_weights_i)
            if(temp>diff):
                diff=temp
        if  (diff<tolerance) :
            delta=False
    return initial_weights         

Using the following parameters, learn the weights on the sales dataset. 

In [187]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
initial_weights = np.zeros(3)
l1_penalty = 1e7
tolerance = 1.0

First create a normalized version of the feature matrix, `normalized_simple_feature_matrix`.

In [188]:
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)
(normalized_simple_feature_matrix, simple_norms) = normalize_features(simple_feature_matrix) # normalize features

Then, run your implementation of LASSO coordinate descent:

In [189]:
l_weights = lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)
print(l_weights)
pred=predict_output(normalized_simple_feature_matrix,l_weights)
err=output-pred
rss=err*err
rss_f=rss.sum()
print(rss_f)

[21624998.81906045 63157246.42159396        0.        ]
1630492484578343.8


***QUIZ QUESTIONS***
1. What is the RSS of the learned model on the normalized dataset? (Hint: use the normalized feature matrix when you make predictions.)
2. Which features had weight zero at convergence?

# Evaluating LASSO fit with more features

Let us split the sales dataset into training and test sets.

In [192]:
train_data=pd.read_csv('NB02_kc_house_train_data.csv', dtype=dtype_dict)
test_data=pd.read_csv('wk5_kc_house_test_data.csv', dtype=dtype_dict)

Let us consider the following set of features.

In [193]:
all_features = ['bedrooms',
                'bathrooms',
                'sqft_living',
                'sqft_lot',
                'floors',
                'waterfront', 
                'view', 
                'condition', 
                'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 
                'yr_renovated']

First, create a normalized feature matrix from the TRAINING data with these features.  (Make you store the norms for the normalization, since we'll use them later)

In [194]:
my_output = 'price'
(all_feature_matrix, output) = get_numpy_data(train_data, all_features, my_output)

(normalized_all_feature_matrix, norms) = normalize_features(all_feature_matrix)

print(normalized_all_feature_matrix)

[[0.00758447 0.00652117 0.0033687  ... 0.         0.00752148 0.        ]
 [0.00758447 0.00652117 0.00757957 ... 0.0057043  0.0075061  0.03707954]
 [0.00758447 0.00434745 0.0033687  ... 0.         0.00743684 0.        ]
 ...
 [0.00758447 0.00652117 0.00842175 ... 0.         0.00772924 0.        ]
 [0.00758447 0.00652117 0.00842175 ... 0.         0.00771    0.        ]
 [0.00758447 0.00434745 0.00252652 ... 0.         0.00772539 0.        ]]


First, learn the weights with `l1_penalty=1e7`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e7`, you will need them later.

In [195]:
initial_weights = np.zeros(14)
l1_penalty = 1e7
tolerance = 1.0
weights1e7= lasso_cyclical_coordinate_descent(normalized_all_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)
print(weights1e7)

[24429601.08756559        0.                0.         48389173.96813957
        0.                0.          3317511.21380402  7329961.8643644
        0.                0.                0.                0.
        0.                0.        ]


***QUIZ QUESTION***

What features had non-zero weight in this case?

Next, learn the weights with `l1_penalty=1e8`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e8`, you will need them later.

In [196]:
initial_weights = np.zeros(14)
l1_penalty = 1e8
tolerance = 1.0
weights1e8= lasso_cyclical_coordinate_descent(normalized_all_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)
print(weights1e8)

[71114625.71488702        0.                0.                0.
        0.                0.                0.                0.
        0.                0.                0.                0.
        0.                0.        ]


***QUIZ QUESTION***

What features had non-zero weight in this case?

Finally, learn the weights with `l1_penalty=1e4`, on the training data. Initialize weights to all zeros, and set the `tolerance=5e5`.  Call resulting weights `weights1e4`, you will need them later.  (This case will take quite a bit longer to converge than the others above.)

In [197]:
initial_weights = np.zeros(14)
l1_penalty = 1e4
tolerance = 5e5
weights1e4= lasso_cyclical_coordinate_descent(normalized_all_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)
print(weights1e4)

[ 78564738.34156765 -22097398.92430532  12791071.87278518
  93808088.09281205  -2013172.75704955  -4219184.9326502
   6482842.81753507   7127408.53480688   5001664.85469637
  14327518.43714046 -15770959.15237402  -5159591.2221315
 -84495341.76843634   2824439.49703684]


***QUIZ QUESTION***

What features had non-zero weight in this case?

## Rescaling learned weights

Recall that we normalized our feature matrix, before learning the weights.  To use these weights on a test set, we must normalize the test data in the same way.

Alternatively, we can rescale the learned weights to include the normalization, so we never have to worry about normalizing the test data: 

In this case, we must scale the resulting weights so that we can make predictions with *original* features:
 1. Store the norms of the original features to a vector called `norms`:
```
features, norms = normalize_features(features)
```
 2. Run Lasso on the normalized features and obtain a `weights` vector
 3. Compute the weights for the original features by performing element-wise division, i.e.
```
weights_normalized = weights / norms
```
Now, we can apply `weights_normalized` to the test data, without normalizing it!

Create a normalized version of each of the weights learned above. (`weights1e4`, `weights1e7`, `weights1e8`).

In [198]:
my_output = 'price'
(all_feature_matrix, output) = get_numpy_data(train_data, all_features, my_output)

(normalized_all_feature_matrix, norms) = normalize_features(all_feature_matrix)
weights1e4_norm=weights1e4/norms
weights1e7_norm=weights1e7/norms
weights1e8_norm=weights1e8/norms

To check your results, if you call `normalized_weights1e7` the normalized version of `weights1e7`, then:
```
print normalized_weights1e7[3]
```
should return 161.31745624837794.

## Evaluating each of the learned models on the test data

Let's now evaluate the three models on the test data:

In [199]:
(test_feature_matrix, test_output) = get_numpy_data(test_data, all_features, 'price')

Compute the RSS of each of the three normalized weights on the (unnormalized) `test_feature_matrix`:

In [200]:
pred1=predict_output(test_feature_matrix,weights1e4_norm)
err1=test_output-pred1
rss1=np.dot(err1,err1)
print(rss1)

228459958971393.28


In [201]:
pred2=predict_output(test_feature_matrix,weights1e7_norm)
err2=test_output-pred2
rss2=np.dot(err2,err2)
print(rss2)

275962077477488.78


In [166]:
pred3=predict_output(test_feature_matrix,weights1e8_norm)
err3=test_output-pred3
rss3=np.dot(err3,err3)
print(rss3)

990207356732103.0


***QUIZ QUESTION***

Which model performed best on the test data?